In [ ]:
%reload_ext autoreload

# Imports

import os
import sys
module_path = os.path.abspath(os.path.join('../image_gen_dm')) # or the path to your source code
sys.path.append(str(module_path))

import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage import gaussian_filter

TF_ENABLE_ONEDNN_OPTS=0
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(TF_ENABLE_ONEDNN_OPTS)
os.environ['TG_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [ ]:
def compare_images(imageA, imageB):
    psnr = PSNR(imageA, imageB)
    ssim = SSIM(imageA, imageB)
    return psnr, ssim

def PSNR(imageA, imageB):
    mse = np.mean((imageA - imageB) ** 2)
    if mse == 0:
        return 100
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def ssim_index(imageA, imageB):
    # Convert images to grayscale based on luminance (Y)
    y = 0.299 * imageA[:, :, 0] + 0.587 * imageA[:, :, 1] + 0.114 * imageA[:, :, 2]
    x = 0.299 * imageB[:, :, 0] + 0.587 * imageB[:, :, 1] + 0.114 * imageB[:, :, 2]

    y = y.astype(np.float64)
    x = x.astype(np.float64)

    # Constants for SSIM index formula
    C1 = (0.01 * 255) ** 2
    C2 = (0.03 * 255) ** 2
    window_size = 11
    sigma = 1.5  # Standard deviation for Gaussian kernel

    # Gaussian window/weights
    window = np.outer(np.hanning(window_size), np.hanning(window_size))
    window /= window.sum()

    # Local means using Gaussian weights
    mu_x = gaussian_filter(x, sigma)
    mu_y = gaussian_filter(y, sigma)

    # Local variance and covariance
    sigma_x2 = gaussian_filter(x**2, sigma) - mu_x**2
    sigma_y2 = gaussian_filter(y**2, sigma) - mu_y**2
    sigma_xy = gaussian_filter(x * y, sigma) - mu_x * mu_y

    # Calculate SSIM index
    num = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
    den = (mu_x**2 + mu_y**2 + C1) * (sigma_x2 + sigma_y2 + C2)
    ssim_map = num / den

    return np.mean(ssim_map)